In [1]:
import gensim.downloader as api

wv = api.load("glove-wiki-gigaword-100")

In [2]:
wv.similarity(w1="great", w2="good")


0.7592797

In [5]:
wv_great = wv["great"]

In [6]:
wv_great.shape

(100,)

In [10]:
import pandas as pd

df = pd.read_csv("Excels\\Fake_Real_Data.csv")
df.shape

(9900, 2)

In [11]:
df.head()

,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake
1,U.S. conservative leader optimistic of common ...,Real
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real
3,Court Forces Ohio To Allow Millions Of Illega...,Fake
4,Democrats say Trump agrees to work on immigrat...,Real


In [12]:
df['label_num'] = df['label'].map({'Fake' : 0, 'Real': 1})
df.head(5)


,Text,label,label_num
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,0
1,U.S. conservative leader optimistic of common ...,Real,1
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,1
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,0
4,Democrats say Trump agrees to work on immigrat...,Real,1


In [27]:
import spacy
nlp = spacy.load("en_core_web_lg")

def preprocess_and_vectorize(text):
    # remove stop words and lemmatize the text
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)
        
    return wv.get_mean_vector(filtered_tokens) # gets the mean of the vectors into 1 sentence embedded vector


In [28]:
v = preprocess_and_vectorize("Don't worry if you don't understand")
v

array([ 0.00824455,  0.1148466 ,  0.10014525, -0.01590417, -0.1867531 ,
       -0.01693138, -0.19055161, -0.13156646,  0.11157142, -0.06287318,
       -0.05655883,  0.04814307,  0.02319344, -0.05732825, -0.08515857,
       -0.0515773 , -0.08774573,  0.10966119, -0.08039526,  0.06311665,
       -0.04348028,  0.05490522, -0.04981112, -0.11757043, -0.07608508,
        0.00596857, -0.01438856, -0.10697965,  0.09737557, -0.02107832,
       -0.04912352,  0.11810335,  0.00292108, -0.02939533, -0.0282766 ,
       -0.00319967, -0.00414034,  0.07270555,  0.11295398, -0.06414554,
       -0.13241655, -0.0130035 , -0.02308624, -0.13509399, -0.10252795,
       -0.02777304,  0.11581741,  0.00577323, -0.08457497, -0.22086534,
        0.1032379 ,  0.01338345, -0.04633459,  0.12219629,  0.06156217,
       -0.29172766,  0.08066311, -0.05254054,  0.16190298,  0.03909582,
       -0.01172133,  0.16293988, -0.05789497, -0.10555087,  0.14668298,
        0.11625271,  0.08954   ,  0.04959325, -0.03425451, -0.04

In [29]:
wv.get_mean_vector(['worry','understand']) # average of both these words vectors

array([ 0.00824455,  0.1148466 ,  0.10014525, -0.01590417, -0.1867531 ,
       -0.01693138, -0.19055161, -0.13156646,  0.11157142, -0.06287318,
       -0.05655883,  0.04814307,  0.02319344, -0.05732825, -0.08515857,
       -0.0515773 , -0.08774573,  0.10966119, -0.08039526,  0.06311665,
       -0.04348028,  0.05490522, -0.04981112, -0.11757043, -0.07608508,
        0.00596857, -0.01438856, -0.10697965,  0.09737557, -0.02107832,
       -0.04912352,  0.11810335,  0.00292108, -0.02939533, -0.0282766 ,
       -0.00319967, -0.00414034,  0.07270555,  0.11295398, -0.06414554,
       -0.13241655, -0.0130035 , -0.02308624, -0.13509399, -0.10252795,
       -0.02777304,  0.11581741,  0.00577323, -0.08457497, -0.22086534,
        0.1032379 ,  0.01338345, -0.04633459,  0.12219629,  0.06156217,
       -0.29172766,  0.08066311, -0.05254054,  0.16190298,  0.03909582,
       -0.01172133,  0.16293988, -0.05789497, -0.10555087,  0.14668298,
        0.11625271,  0.08954   ,  0.04959325, -0.03425451, -0.04

In [30]:
df = df[:1000]
df.shape

(1000, 4)

In [31]:
df['vector'] = df['Text'].apply(lambda text: preprocess_and_vectorize(text))
df.head()

,Text,label,label_num,vector
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,0,"[-0.015056321, 0.05181379, 0.07991907, -0.0393..."
1,U.S. conservative leader optimistic of common ...,Real,1,"[-0.032683548, 0.04444601, 0.027827501, -6.240..."
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,1,"[-0.009751513, 0.04463293, 0.03933768, -0.0186..."
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,0,"[-0.015562437, 0.015744263, 0.055174056, -0.04..."
4,Democrats say Trump agrees to work on immigrat...,Real,1,"[-0.009034788, 0.021228185, 0.02940816, -0.012..."


In [32]:
from sklearn.model_selection import train_test_split


#Do the 'train-test' splitting with test size of 20% with random state of 2022 and stratify sampling too
X_train, X_test, y_train, y_test = train_test_split(
    df.vector.values, 
    df.label_num, 
    test_size=0.2, # 20% samples will go to test dataset
    random_state=2022,
    stratify=df.label_num
)

In [ ]:
X_train[:2]

In [ ]:
print("Shape of X_train before reshaping: ", X_train.shape)
print("Shape of X_test before reshaping: ", X_test.shape)


X_train_2d = np.stack(X_train)
X_test_2d =  np.stack(X_test)

print("Shape of X_train after reshaping: ", X_train_2d.shape)
print("Shape of X_test after reshaping: ", X_test_2d.shape)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

#1. creating a GradientBoosting model object
clf = GradientBoostingClassifier()

#2. fit with all_train_embeddings and y_train
clf.fit(X_train_2d, y_train)


#3. get the predictions for all_test_embeddings and store it in y_pred
y_pred = clf.predict(X_test_2d)


#4. print the classfication report
print(classification_report(y_test, y_pred))

In [ ]:
test_news = [
    "Michigan governor denies misleading U.S. House on Flint water (Reuters) - Michigan Governor Rick Snyder denied Thursday that he had misled a U.S. House of Representatives committee last year over testimony on Flintâ€™s water crisis after lawmakers asked if his testimony had been contradicted by a witness in a court hearing. The House Oversight and Government Reform Committee wrote Snyder earlier Thursday asking him about published reports that one of his aides, Harvey Hollins, testified in a court hearing last week in Michigan that he had notified Snyder of an outbreak of Legionnairesâ€™ disease linked to the Flint water crisis in December 2015, rather than 2016 as Snyder had testified. â€œMy testimony was truthful and I stand by it,â€ Snyder told the committee in a letter, adding that his office has provided tens of thousands of pages of records to the committee and would continue to cooperate fully.  Last week, prosecutors in Michigan said Dr. Eden Wells, the stateâ€™s chief medical executive who already faced lesser charges, would become the sixth current or former official to face involuntary manslaughter charges in connection with the crisis. The charges stem from more than 80 cases of Legionnairesâ€™ disease and at least 12 deaths that were believed to be linked to the water in Flint after the city switched its source from Lake Huron to the Flint River in April 2014. Wells was among six current and former Michigan and Flint officials charged in June. The other five, including Michigan Health and Human Services Director Nick Lyon, were charged at the time with involuntary manslaughter",
    " WATCH: Fox News Host Loses Her Sh*t, Says Investigating Russia For Hacking Our Election Is Unpatriotic This woman is insane.In an incredibly disrespectful rant against President Obama and anyone else who supports investigating Russian interference in our election, Fox News host Jeanine Pirro said that anybody who is against Donald Trump is anti-American. Look, it s time to take sides,  she began.",
    " Sarah Palin Celebrates After White Man Who Pulled Gun On Black Protesters Goes Unpunished (VIDEO) Sarah Palin, one of the nigh-innumerable  deplorables  in Donald Trump s  basket,  almost outdid herself in terms of horribleness on Friday."
]

test_news_vectors = [preprocess_and_vectorize(n) for n in test_news]
clf.predict(test_news_vectors)